In [ ]:
!pip install https://github.com/matplotlib/mpl_finance/archive/master.zip

!pip install mplfinance

!pip install plotly>=4.0.0
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from openpyxl import load_workbook
import csv
import cv2
import plotly.graph_objects as go
import mplfinance as mpf
import pandas as pd
import matplotlib.dates as mpl_dates
import numpy as np
import os
import imageio

In [ ]:
# Use mplfinance API for create candlestick image
# github link: https://github.com/matplotlib/mplfinance
def plot3(file,saveAs):
  daily = pd.read_csv(file,index_col=0,parse_dates=True)
  daily.index.name = 'Date'
  mc = mpf.make_marketcolors(up='green', down='red', edge='inherit', wick='inherit')
  ms = mpf.make_mpf_style(marketcolors=mc)
  mpf.plot(daily, type='candle', savefig=saveAs, figsize=(0.56,0.56), style=ms, ylabel='', axisoff=True, scale_padding=0.0, scale_width_adjustment=dict(candle=0.8))

def windowClass(ws):
  maxrow = 42831
  window = []
  for r in range(2, maxrow + 1):
    window.append(ws[f'A{r}'].value)
  return window

def dateClass(ws):
  maxrow = 42831
  date = []
  for r in range(2, maxrow + 1):
    date.append(ws[f'A{r}'].value)
  return date

def openClass(ws):
  maxrow = 42831
  open = []
  for r in range(2, maxrow + 1):
    open.append(ws[f'B{r}'].value)
  return open

def closeClass(ws):
  maxrow = 42831
  close = []
  for r in range(2, maxrow + 1):
    close.append(ws[f'E{r}'].value)
  return close

def highClass(ws):
  maxrow = 42831
  high = []
  for r in range(2, maxrow + 1):
    high.append(ws[f'C{r}'].value)
  return high

def lowClass(ws):
  maxrow = 42831
  low = []
  for r in range(2, maxrow + 1):
    low.append(ws[f'D{r}'].value)
  return low

def volumeClass(ws):
  maxrow = 42831
  low = []
  for r in range(2, maxrow + 1):
    low.append(ws[f'F{r}'].value)
  return low

# This function can read a excel file and translate into candlestick images
# Those images will separated into two folders, which represnt historical and future
def xlsxToImage(loc, save_loc1, save_loc2, sheet, first, last, id):
  wb = load_workbook(filename=loc)
  ws = wb[sheet]
  #window=windowClass(ws)
  date=dateClass(ws)
  openPrice=openClass(ws)
  high=highClass(ws)
  low=lowClass(ws)
  close=closeClass(ws)
  volume=volumeClass(ws)
  # Folders locations cna be changed here
  os.makedirs(f'drive/MyDrive/GanStick/dataset_eq/{save_loc1}/{sheet}', exist_ok=True)
  os.makedirs(f'drive/MyDrive/GanStick/dataset_eq/{save_loc2}/{sheet}', exist_ok=True)
  for i in range(first, last-9, 1):
    with open(f'data_{i}.csv', 'w') as file:
      writer = csv.writer(file)
      writer.writerow(["Date", "Open", "High", "Low", "Close", "Volume"])
      writer.writerow([date[i], openPrice[i], high[i], low[i], close[i], volume[i]])
      writer.writerow([date[i + 1], openPrice[i + 1], high[i + 1], low[i + 1], close[i + 1], volume[i+1]])
      writer.writerow([date[i + 2], openPrice[i + 2], high[i + 2], low[i + 2], close[i + 2], volume[i+2]])
      writer.writerow([date[i + 3], openPrice[i + 3], high[i + 3], low[i + 3], close[i + 3], volume[i+3]])
      writer.writerow([date[i + 4], openPrice[i + 4], high[i + 4], low[i + 4], close[i + 4], volume[i+4]])
    plot3(f'data_{i}.csv',f'drive/MyDrive/GanStick/dataset_eq/{save_loc1}/{sheet}/{sheet}_{id:05}.png')
    file.close()
    os.remove(f'data_{i}.csv')

    j = i+5
    with open(f'data_{j}.csv', 'w') as file:
      writer = csv.writer(file)
      writer.writerow(["Date", "Open", "High", "Low", "Close", "Volume"])
      writer.writerow([date[j], openPrice[j], high[j], low[j], close[j], volume[j]])
      writer.writerow([date[j + 1], openPrice[j + 1], high[j + 1], low[j + 1], close[j + 1], volume[j+1]])
      writer.writerow([date[j + 2], openPrice[j + 2], high[j + 2], low[j + 2], close[j + 2], volume[j+2]])
      writer.writerow([date[j + 3], openPrice[j + 3], high[j + 3], low[j + 3], close[j + 3], volume[j+3]])
      writer.writerow([date[j + 4], openPrice[j + 4], high[j + 4], low[j + 4], close[j + 4], volume[j+4]])
    plot3(f'data_{j}.csv',f'drive/MyDrive/GanStick/dataset_eq/{save_loc2}/{sheet}/{sheet}_{id:05}.png')
    file.close()
    os.remove(f'data_{j}.csv')

    id += 1

In [ ]:
# Following are the function calls for creating candlestick images
# xlsxToImage(excel_file_location, historical_folder_name, future_folder_name, sheet_name, from, to, first_index)
# from: The first data index we want in excel file. Since some data are lost in google finance, we need to set up this index.
# to: The last data index we want in excel file.
# first_index: Sometimes, colab might disconnect in the middle of translating. This config can help you start the function in the middle.

#print("Amazon")
#xlsxToImage('drive/MyDrive/GanStick/Stock.xlsx', 'historical', 'future', 'Amazon', 0, 5695, 1)
#print("Apple")
#xlsxToImage('drive/MyDrive/GanStick/Stock.xlsx', 'historical', 'future', 'Apple', 2567, 8571, 1)
#print("BOA")
#xlsxToImage('drive/MyDrive/GanStick/Stock.xlsx', 'historical', 'future', 'BOA', 2566, 8570, 1)
#print("Google")
#xlsxToImage('drive/MyDrive/GanStick/Stock.xlsx', 'historical', 'future', 'Google', 0, 3868, 1)
#print("J&J")
#xlsxToImage('drive/MyDrive/GanStick/Stock.xlsx', 'historical', 'future', 'JJ', 2566, 8570, 1)
#print("JP_Morgan")
#xlsxToImage('drive/MyDrive/GanStick/Stock.xlsx', 'historical', 'future', 'JP', 0, 8569, 1)
#print("Microsoft")
#xlsxToImage('drive/MyDrive/GanStick/Stock.xlsx', 'historical', 'future', 'Microsoft', 2470, 8521, 1)
#print("P&G")
#xlsxToImage('drive/MyDrive/GanStick/Stock.xlsx', 'historical', 'future', 'PG', 2566, 8570, 1)
#print("United_Healthcare")
#xlsxToImage('drive/MyDrive/GanStick/Stock.xlsx', 'historical', 'future', 'UH', 2566, 8571, 1)
#print("Visa")
#xlsxToImage('drive/MyDrive/GanStick/Stock.xlsx', 'historical', 'future', 'Visa', 0, 2968, 1)
#print("Walmart")
#xlsxToImage('drive/MyDrive/GanStick/Stock.xlsx', 'historical', 'future', 'Walmart', 2565, 8569, 1)

In [ ]:
#xlsxToImage('drive/MyDrive/GanStick/sector_hist/abbv.xlsx', 'sector/historical', 'sector/future', 'abbv', 0, 2199, 1)
#xlsxToImage('drive/MyDrive/GanStick/sector_hist/amt.xlsx', 'sector/historical', 'sector/future', 'amt', 0, 5934, 1)
#xlsxToImage('drive/MyDrive/GanStick/sector_hist/bac.xlsx', 'sector/historical', 'sector/future', 'bac', 3313, 12217, 1)
#xlsxToImage('drive/MyDrive/GanStick/sector_hist/bhp.xlsx', 'sector/historical', 'sector/future', 'bhp', 1920, 10447, 1)
#xlsxToImage('drive/MyDrive/GanStick/sector_hist/cci.xlsx', 'sector/historical', 'sector/future', 'cci', 0, 5815, 1)
#xlsxToImage('drive/MyDrive/GanStick/sector_hist/cvx.xlsx', 'sector/historical', 'sector/future', 'cvx', 1987, 15036, 1)
#xlsxToImage('drive/MyDrive/GanStick/sector_hist/duk.xlsx', 'sector/historical', 'sector/future', 'duk', 773, 10472, 1)
xlsxToImage('drive/MyDrive/GanStick/sector_hist/etr.xlsx', 'sector/historical', 'sector/future', 'etr', 1, 12437, 1)